# IMPORTS

In [1]:
!pip install -q git+https://github.com/tensorflow/docs 

In [2]:
pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#############
## Imports ##
#############

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')


STYLE = "#ffffff"

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time
import random
import h5py

import tensorflow as tf
from tensorflow.keras import Sequential, layers
import keras

from scipy.fft import dst, dct, fft

from IPython import display

# VARIABLES

In [5]:
##############################
## Define the training loop ##
##############################

load_data = True

n_samples = 5000

void_dim = 56

BUFFER_SIZE = 5000

BATCH_SIZE = 32

EPOCHS = 500

noise_dim = 1024

num_examples_to_generate = 1

seed = tf.random.normal([num_examples_to_generate, noise_dim])

# You will reuse this seed overtime (so it's easier) to visualize progress in the animated GIF)

# ELEMENTS CREATOR

In [6]:
class Sculptor():
    
    def __init__(self, void_dim,
                 n_edge_elements,
                 n_plane_elements,
                 n_volume_elements,
                 element_edge_min,
                 element_edge_max,
                 element_plane_min,
                 element_plane_max,
                 element_volume_min,
                 element_volume_max,
                 step,
                 verbose):
        
        self.void = np.zeros((void_dim, void_dim, void_dim))
        self.n_edge_elements = n_edge_elements
        self.n_plane_elements = n_plane_elements
        self.n_volume_elements = n_volume_elements
        self.style = "#ffffff"
        
        self.element_edge_min= element_edge_min
        self.element_edge_max = element_edge_max
        self.element_plane_min = element_plane_min
        self.element_plane_max = element_plane_max
        self.element_volume_min = element_volume_min
        self.element_volume_max = element_volume_max
        self.step = step
        
        self.verbose = verbose
        
        
    def return_axis(self):
        
        self.section = np.random.randint(low=0-1, high=self.void[0].shape[0])
        self.axis_selection = np.random.randint(low=0, high=3)
        
        if self.axis_selection == 0:
            self.working_plane = self.void[self.section,:,:]
        elif self.axis_selection == 1:
            self.working_plane = self.void[:,self.section,:]  
        elif self.axis_selection == 2:
            self.working_plane = self.void[:,:,self.section]
        else:
            print("error")
        return self.working_plane
    
    ### MAIN FUNCTIONS ###
    
    def add_edge(self): # element sizes
        
        self.working_plane = self.return_axis()
        # selection of the axis to work on
        if self.verbose == True:
            print(working_plane)
            print("###############################################################")

        #Variables
        self.edge_length = random.randrange(self.element_edge_min, self.element_edge_max, self.step) # estas variables quizas no necesiten ser self!!
        self.edge_plane = np.random.randint(low=0, high=2)

        if self.edge_plane == 0:
            self.element = np.ones(self.edge_length).reshape(self.edge_length,1)
        else:
            self.element = np.ones(self.edge_length).reshape(self.edge_length,1).T

        # creates the element to be inserted
        self.delta = np.array(self.working_plane.shape) - np.array(self.element.shape) 
        # finds the delta between the size of the void and the size of the element
        top_left_corner = (coor_i, coor_j) = (np.random.randint(low=0, high=self.delta[0]) , np.random.randint(low=0, high=self.delta[1]))
        # finds the coordinates of the top left corner
        top_left_corner = np.array(top_left_corner)
        # converts the result in an array
        bottom_right_corner = np.array(top_left_corner) + np.array(self.element.shape) #- np.array([1,1]))
        # finds the coordinates of the bottom right corner
        self.working_plane[top_left_corner[0]:bottom_right_corner[0] , top_left_corner[1]:bottom_right_corner[1]] = self.element
        # makes the slides using the coordinates equal to the element

        if self.verbose == True:
            print(self.working_plane)
            print("###############################################################")
    
    def add_plane(self): # element sizes
        
        self.element = None
        self.section = None
        self.delta = None
        self.top_left_corner = None
        self.bottom_right_corner = None
        self.working_plane = self.return_axis()
        if self.verbose == True:
            print(self.working_plane)
            print("###############################################################")

        #Variables
        self.element = np.ones((random.randrange(self.element_plane_min, self.element_plane_max, self.step), random.randrange(self.element_plane_min, self.element_plane_max, self.step)))
        # creates the element to be inserted
        self.delta = np.array(self.working_plane.shape) - np.array(self.element.shape) 
        # finds the delta between the size of the void and the size of the element
        self.top_left_corner = (coor_i, coor_j) = (np.random.randint(low=0, high=self.delta[0]) , np.random.randint(low=0, high=self.delta[1]))
        # finds the coordinates of the top left corner
        self.top_left_corner = np.array(self.top_left_corner)
        # converts the result in an array
        self.bottom_right_corner = np.array(self.top_left_corner) + np.array(self.element.shape) #- np.array([1,1]))
        # finds the coordinates of the bottom right corner
        self.working_plane[self.top_left_corner[0]:self.bottom_right_corner[0] , self.top_left_corner[1]:self.bottom_right_corner[1]] = self.element
        # makes the slides using the coordinates equal to the element

        if self.verbose == True:
            self.print_information()
            print("###############################################################")
            
        return self.void
    
    def add_pipe_cantilever(self):
        
        self.element = None
        self.working_plane = None
        self.delta = None
        self.top_left_corner = None
        self.bottom_right_corner = None
        self.axis_selection = np.random.randint(low=0, high=2)
        self.shape_selection = np.random.randint(low=0, high=2)
        self.depth = random.randrange(self.element_volume_min, self.element_volume_max, self.step)
        
        if self.verbose == True:
            print(self.working_plane)
            print("###############################################################")
            
        self.element = np.ones((random.randrange(self.element_volume_min, self.element_volume_max, self.step), random.randrange(self.element_volume_min, self.element_volume_max, self.step)))
        self.element = np.repeat(self.element, repeats=self.depth, axis=0).reshape((self.element.shape[0],self.element.shape[1],self.depth))

        self.element_void = np.zeros((self.element.shape[0]-2, self.element.shape[1]-2))
        self.element_void = np.repeat(self.element_void, repeats=self.depth).reshape((self.element_void.shape[0],self.element_void.shape[1],self.depth))

        # element[1:-1,1:-1,:] = element_void # elegir pasar el vacio o no como parte del volumen
        
        self.delta = np.array(self.void.shape) - np.array(self.element.shape) # ENCONTRAR LOS NUEVOS DELTAS

        corner_1 = np.array((np.random.randint(low=0, high=self.delta[0]) , np.random.randint(low=0, high=self.delta[1]), np.random.randint(low=0, high=self.delta[2])))
        corner_2 = np.array((corner_1[0] + self.element.shape[0], corner_1[1], corner_1[2]))
        corner_3 = np.array((corner_1[0], corner_1[1], corner_1[2] + self.element.shape[2]))
        corner_4 = np.array((corner_1[0] + self.element.shape[0], corner_1[1], corner_1[2] + self.element.shape[2]))
        
        corner_5 = np.array((corner_1[0], corner_1[1] + self.element.shape[1], corner_1[2]))
        corner_6 = np.array((corner_2[0], corner_2[1] + self.element.shape[1], corner_2[2]))
        corner_7 = np.array((corner_3[0], corner_3[1] + self.element.shape[1], corner_3[2]))
        corner_8 = np.array((corner_4[0], corner_4[1] + self.element.shape[1], corner_4[2]))
        
        # creates the floor and ceiling
        self.void[corner_3[0]:corner_8[0], corner_3[1]:corner_8[1], corner_3[2]-1] = self.element[:,:,0]
        self.void[corner_1[0]:corner_6[0], corner_1[1]:corner_6[1], corner_1[2]] = self.element[:,:,1]
        
        # creates de walls
        if self.shape_selection ==0:
            if self.axis_selection == 0:
                self.void[corner_1[0], corner_1[1]:corner_7[1], corner_1[2]:corner_7[2]] = self.element[0,:,:]
                self.void[corner_2[0]-1, corner_2[1]:corner_8[1], corner_2[2]:corner_8[2]] = self.element[1,:,:]
            else:
                self.void[corner_5[0]:corner_8[0], corner_5[1], corner_5[2]:corner_8[2]] = self.element[:,0,:]
                self.void[corner_1[0]:corner_4[0], corner_1[1], corner_1[2]:corner_4[2]] = self.element[:,0,:]
                
        else:
            if self.axis_selection == 0:
                self.void[corner_1[0], corner_1[1]:corner_7[1], corner_1[2]:corner_7[2]] = self.element[0,:,:]
                self.void[corner_5[0]:corner_8[0], corner_5[1], corner_5[2]:corner_8[2]] = self.element[:,0,:]
            else:
                self.void[corner_2[0]-1, corner_2[1]:corner_8[1], corner_2[2]:corner_8[2]] = self.element[1,:,:]
                self.void[corner_1[0]:corner_4[0], corner_1[1], corner_1[2]:corner_4[2]] = self.element[:,0,:]
        
        if self.verbose == True:
            self.print_information()
            print("###############################################################")
        
        return self.void
    
    def add_grill(self):
        pass
    
    ### ULTILS ###
    
    def print_information(self):
        print(f"void shape is: {np.array(self.void[0].shape)}")
        print(f"element shape is : {np.array(self.element.shape)}")
        print(f"the axis selection is: {self.axis_selection}")
        print(f"delta is: {self.delta}")
        print(f"section is: {self.section}")
        print(f"top left corner is: {self.top_left_corner}")
        print(f"bottom right corner is: {self.bottom_right_corner}")
        print(f"slices are: {self.top_left_corner[0]}:{self.bottom_right_corner[0]} and {self.top_left_corner[1]}:{self.bottom_right_corner[1]}")
        print("###############################################################")
    
    def plot_sections(self):
        sculpture = self.void
        fig, axes = plt.subplots(ncols=6, nrows=int(np.ceil(self.void.shape[0]/6)), figsize=(25, 25), facecolor = (self.style))
        axes = axes.ravel() # flats
        for index in range(self.void.shape[0]):
            axes[index].imshow(sculpture[index,:,:], cmap = "gray")
            
    def plot_sculpture(self):
        sculpture = self.void
        fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(25, 25), facecolor = (self.style), subplot_kw=dict(projection="3d"))
        axes = axes.ravel() # flats
        for index in range(1):
            axes[index].voxels(sculpture, facecolors="orange", edgecolors="k", linewidth=0.05)
    
    ### GENERATOR ###
    
    def generative_sculpt(self):
        for i in range(self.n_edge_elements):
            time.sleep(0)
            self.axis_selection = np.random.randint(low=0, high=3)
            self.add_edge()
            
        for i in range(self.n_plane_elements):
            time.sleep(0)
            self.axis_selection = np.random.randint(low=0, high=3)
            self.add_plane()
            
        for i in range(self.n_volume_elements):
            self.add_pipe_cantilever()

        return self.void

# DATA

In [7]:
if load_data:
  os.chdir("/content/drive/MyDrive/data/volumetries")

  raw_data = np.load("raw-data-new.npy")

else:
  os.chdir("/content/drive/MyDrive/data/volumetries")
  raw_data = []
  count = 0

  for sculpture in range(n_samples): #
      count = count + 1
      if count % 10 == 0:
          print("\r{0}".format(count), end='')
      
      sculptor = Sculptor(void_dim = void_dim,
                    n_edge_elements = 0,
                    n_plane_elements = 8,
                    n_volume_elements = 3,
                    element_edge_min= 16,
                    element_edge_max = 48,
                    element_plane_min = 16,
                    element_plane_max = 30,
                    element_volume_min = 10,
                    element_volume_max = 30,
                    step = 3,
                    verbose = False)
      
      sculpture = sculptor.generative_sculpt()
      
      raw_data.append(sculpture)

  raw_data = np.asarray(raw_data).reshape((n_samples, void_dim, void_dim, void_dim, 1))

  np.save("raw-data-new", raw_data, allow_pickle=False)

# MODEL GENERATOR

In [8]:
# The generator uses tf.keras.layers.Conv2DTranspose (upsampling) layers to produce an image from a seed (random noise). 

def make_three_dimentional_generator():
    model = tf.keras.Sequential() # Initialize Sequential model
    # The Sequential model is a straight line. You keep adding layers, every new layer takes the output of the previous layer. You cannot make creative graphs with branches
    # The functoinal API Model is completely free to have as many ramifications, inputs and outputs as you need
    model.add(layers.Dense(7*7*7*256, use_bias=False, input_shape=(noise_dim,))) # shape 100 noise vector, 7*7*256 flat layer to reshape [7,7,256] | 7 width 7 height 256 channels
    model.add(layers.BatchNormalization()) # BatchNormalization doesn't require bias, makes the model faster and more stable
    model.add(layers.ReLU()) # LeakyReLU
    model.add(layers.Reshape((7, 7, 7, 256))) # reshape [7,7,256] 
    assert model.output_shape == (None, 7, 7, 7, 256) # None is the batch size

    model.add(layers.Conv3DTranspose(128, (9, 9, 9), strides=(1, 1, 1), padding='same', use_bias=False)) # 128 Filters... to be the number of channels of the output, (5,5) kernel
    assert model.output_shape == (None, 7, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv3DTranspose(64, (7, 7, 7), strides=(2, 2, 2), padding='same', use_bias=False)) # 128 Filters... to be the number of channels of the output, (5,5) kernel
    assert model.output_shape == (None, 14, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv3DTranspose(32, (7, 7, 7), strides=(2, 2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 28, 28, 28, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv3DTranspose(1, (5, 5, 5), strides=(2, 2, 2), padding='same', use_bias=False, activation='tanh'))
    model.add(layers.ThresholdedReLU(theta = 0))
    assert model.output_shape == (None, 56, 56, 56, 1)
    return model

generator = make_three_dimentional_generator()

# MODEL CRITIC

In [9]:
# The discriminator is a CNN-based image classifier it uses tf.keras.layers.Conv2D to classify images as real or fake

def make_three_dimentional_critic():
    model = tf.keras.Sequential()
    model.add(layers.Conv3D(32, (5, 5, 5), strides=(2, 2, 2), padding='same', input_shape=[56, 56, 56, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv3D(64, (5, 5, 5), strides=(2, 2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv3D(128, (5, 5, 5), strides=(2, 2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

discriminator = make_three_dimentional_critic()

In [10]:
decision = discriminator(generator(tf.random.normal([1, noise_dim])))#[0].reshape((28,28,28,1)))

print(decision) # 50 / 50 not trained, will be trained to generate positive values for real pictures and negative for generated ones

tf.Tensor([[0.4999786]], shape=(1, 1), dtype=float32)


# SUMMARIES

In [11]:
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 87808)             89915392  
                                                                 
 batch_normalization (BatchN  (None, 87808)            351232    
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 87808)             0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 7, 256)      0         
                                                                 
 conv3d_transpose (Conv3DTra  (None, 7, 7, 7, 128)     23887872  
 nspose)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 7, 128)     5

In [12]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 28, 28, 32)    4032      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 28, 28, 28, 32)    0         
                                                                 
 dropout (Dropout)           (None, 28, 28, 28, 32)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 14, 14, 14, 64)    256064    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 14, 64)    0         
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 14, 64)    0         
                                                                 
 conv3d_2 (Conv3D)           (None, 7, 7, 7, 128)     

# LOSS

In [13]:
##################################
## Model Compile: Loss Function ##
##################################

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True) # we take a BinaryCrossentropy

# Binary cross entropy compares each of the predicted probabilities to actual class output which can be either 0 or 1
# Binary Cross Entropy is the negative average of the log of corrected predicted probabilities

########################
## Discriminator loss ##
########################

# quantifies how well the discriminator is able to distinguish real images from generated 

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    # compares the predictions of the discriminator over real images to a matrix of [1s] | must have a tendency/likelihood to 1
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    # compares the predictions of the discriminator over generated images to a matrix of [0s] | must have a tendency/likelihood to 0
    total_loss = real_loss + fake_loss
    return total_loss # Total loss

####################
## Generator loss ##
####################

# quantifies how well it was able to trick the discriminator, if the generator is performing well, the discriminator will classify the fake images as real (1). 

def generator_loss(fake_output):
    binary_cross_entropy = cross_entropy(tf.ones_like(fake_output), fake_output)
    # the generator's output need to have a tendency to 1, We compare the discriminators decisions on the generated images to an array of [1s]
    return binary_cross_entropy

# OPTIMIZER

In [14]:
##############################
## Model Compile: Optimizer ##
##############################

# Two different optimizers since we train two separate networks:

generator_optimizer = tf.keras.optimizers.Adam(1e-4) # SGD INSTEAD???   (Radford et al., 2015)

discriminator_optimizer = tf.keras.optimizers.Adam(1e-4) # SGD INSTEAD???  (Radford et al., 2015)

# CHECKPOINT

In [15]:
######################
## Save checkpoints ##
######################

# save and restore models, which can be helpful in case a long running training task is interrupted.

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 #discriminator_optimizer=discriminator_optimizer,
                                 generator=generator) #,
                                 #discriminator=discriminator)

checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 #discriminator_optimizer=discriminator_optimizer,
                                 generator=generator) #,
                                 #discriminator=discriminator)


# TRAINING LOOP

In [16]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function

####################
## Training steps ##
####################

def train_step(images): # train for just ONE STEP aka one forward and back propagation

    noise = tf.random.normal([BATCH_SIZE, noise_dim]) #tf.random.normal([BATCH_SIZE, noise_dim]) # generate the noises [batch size, latent space 100 dimention vector]

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape: # get the gradient for each parameter for this step
      generated_images = generator(noise, training=True) # iterates over the noises

      real_output = discriminator(images, training=True) # trains discriminator based on labeled real pics
      fake_output = discriminator(generated_images, training=True) # trains discriminator based on labeled generated pics
    # why it doesnt traing all at ones

      gen_loss = generator_loss(fake_output) # calculating the generator loss function previously defined
      disc_loss = discriminator_loss(real_output, fake_output) # calculating the descrim loss function previously defined

    print(f"gen loss : {gen_loss}")
    print(f"gen loss : {disc_loss}")

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    # saving the gradients of each trainable variable of the generator
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    # saving the gradients of each trainable variable of the discriminator

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    # applying the gradients on the trainable variables of the generator to update the parameters
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    # applying the gradients on the trainable variables of the generator to update the parameters



In [17]:
###################
## Training loop ##
###################

# training loop itself using train_step function previously defined 

def train(dataset, epochs):

  # load checkpoint
  
  for epoch in range(epochs):

    start = time.time()
    
    print("\nStart of epoch %d" % (epoch+1,))
    print("################################")
    
    for index, image_batch in enumerate(dataset):
        noise = tf.random.normal([BATCH_SIZE, noise_dim]) #tf.random.normal([BATCH_SIZE, noise_dim]) # generate the noises [batch size, latent space 100 dimention vector]

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape: # get the gradient for each parameter for this step
          generated_images = generator(noise, training=True) # iterates over the noises

          real_output = discriminator(image_batch, training=True) # trains discriminator based on labeled real pics
          fake_output = discriminator(generated_images, training=True) # trains discriminator based on labeled generated pics
        # why it doesnt traing all at ones

          gen_loss = generator_loss(fake_output) # calculating the generator loss function previously defined
          disc_loss = discriminator_loss(real_output, fake_output) # calculating the descrim loss function previously defined
        
        if (index + 1) % 25 == 0:
          print(f"Minibatch Number {index+1}")
          print(f"The loss of the generator is: {gen_loss}") #, end="\r")
          print(f"The loss of the discriminator is: {disc_loss}") #, end="\r")
          print("################################") #, end="\r")

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        # saving the gradients of each trainable variable of the generator
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        # saving the gradients of each trainable variable of the discriminator

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        # applying the gradients on the trainable variables of the generator to update the parameters
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        # applying the gradients on the trainable variables of the generator to update the parameters

    # Produce images 
    display.clear_output(wait=True) # clearing output !!!TO BE CHECKED!!!
    # generate_and_save_images(generator, epoch + 1, seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 5 == 0:
      os.chdir("/content/drive/MyDrive/data/volumetries")
      checkpoint.save(file_prefix = checkpoint_prefix) # saving weights and biases previously calculated by the train step gradients
    if (epoch + 1) % 5 == 0:  
      generate_and_save_images(generator, epoch + 1, seed)
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  # generate_and_save_images(generator, epochs, seed)

In [18]:
predictions = generator(seed, training=False)

predictions.shape

TensorShape([1, 56, 56, 56, 1])

# GENERATE AND SAVE

In [19]:
###############################
## Generate and save images  ##
###############################

def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  os.chdir("/content/drive/MyDrive/data/volumetries/images")

  predictions = model(test_input, training=False)

  fig, axes = plt.subplots(ncols=1, nrows=1, figsize=(25, 25), facecolor = (STYLE), subplot_kw=dict(projection="3d"))
  
  axes.voxels(predictions[0,:,:,:,0], facecolors="orange", edgecolors="k", linewidth=0.05)

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))

#TRAINING

In [20]:
if load_data:
  os.chdir("/content/drive/MyDrive/data/volumetries")
  train_dataset = tf.data.Dataset.from_tensor_slices(raw_data[:4975]).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
else:
  os.chdir("/content/drive/MyDrive/data/volumetries")
  raw_data = np.load("/content/drive/MyDrive/data/volumetries/raw-data-new.npy")
  train_dataset = tf.data.Dataset.from_tensor_slices(raw_data[:4975]).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
train(train_dataset, EPOCHS)


Start of epoch 1
################################


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Minibatch Number 25
The loss of the generator is: 0.8391766548156738
The loss of the discriminator is: 0.9114106893539429
################################


# ITERPOLATIONS & ANIMATIONS

In [ ]:
checkpoint.restore("/content/drive/MyDrive/data/volumetries/images/training_checkpoints/ckpt-13.index")

In [ ]:
noise = tf.random.normal([2, 1, noise_dim]) # Random input vecto [number of samples, Width, Height]

x = noise[0, 0, :].numpy()

y = noise[1, 0, :].numpy()

In [ ]:
# uniform interpolation between two points in latent space
def interpolate_points(p1, p2, n_steps=100):
	# interpolate ratios between the points
	ratios = np.linspace(0, 1, num=n_steps)
	# linear interpolate vectors
	vectors = list()
	for ratio in ratios:
		v = (1.0 - ratio) * p1 + ratio * p2
		vectors.append(v)
	return np.asarray(vectors)
 
interpolation_result = interpolate_points(x, y, n_steps=75)

In [ ]:
interpolation_result.shape

In [ ]:
os.chdir("/content/drive/MyDrive/data/volumetries/images")

count = 1

for index, restult in enumerate(interpolation_result):
  start = time.time()
  print(f"generating frame number {count}")
  fig, axes = plt.subplots(ncols=1, nrows=1, figsize=(25, 25), facecolor = (STYLE), subplot_kw=dict(projection="3d"))

  for plot in range(1):
    axes.voxels(generator(result[plot].reshape((1,512)), training=False)[0, :, :, :, 0], facecolors="orange", edgecolors="k", linewidth=0.05)
    plt.savefig('image_interpolation_{:04d}.png'.format(count))
  print ('Time for frame {} is {} sec'.format(index + 1, time.time()-start))
  count = count + 1

In [ ]:
# plt.plot(interpolation_result[:,0:1], result[:,1:2])

# plt.show

In [ ]:
############################
## Display a single image ##
############################

def display_image(epoch_no): # using the epoch number
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

In [ ]:
##################
## Animated GIF ##
##################

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image_inter*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed

embed.embed_file(anim_file)

# BONUS AND TRASH

In [ ]:
param_setters = dict()
for var in tf.trainable_variables():
    placeholder = tf.placeholder(var.dtype, var.shape, var.name.split(':')[0]+'_setter')
    param_setters[var.name] = (tf.assign(var, placeholder), placeholder)

In [ ]:
generator.load_weights("/content/drive/MyDrive/data/volumetries/training_checkpoints/checkpoint")

In [ ]:
# os.chdir("/content/drive/MyDrive/data/volumetries/training_checkpoints")

# reader = tf.train.load_checkpoint('/content/drive/MyDrive/data/volumetries/training_checkpoints/ckpt-19.data-00000-of-00001')
# shape_from_key = reader.get_variable_to_shape_map()
# dtype_from_key = reader.get_variable_to_dtype_map()

# sorted(shape_from_key.keys())

In [ ]:
# save_path = "/content/drive/MyDrive/data/volumetries/training_checkpoints/ckpt-19.data-00000-of-00001"

# tf.train.Checkpoint.read(save_path=save_path, options=None)

In [ ]:
# with h5py.File('model.hdf5', 'w') as f:
#     for var in tf.trainable_variables():
#         key = var.name.replace('/', ' ')
#         value = session.run(var)
#         f.create_dataset(key, data=value)

# tensor flow lo tiene como opcion

In [ ]:
# with h5py.File('model.hdf5', 'w') as f:
#     for var in tf.trainable_variables():
#         key = var.name.replace('/', ' ')
#         value = session.run(var)
#         f.create_dataset(key, data=value)

In [ ]:
# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# # Train the model with the new callback
# model.fit(train_images, 
#           train_labels,  
#           epochs=10,
#           validation_data=(test_images, test_labels),
#           callbacks=[cp_callback])  # Pass callback to training

# # This may generate warnings related to saving the state of the optimizer.
# # These warnings (and similar warnings throughout this notebook)
# # are in place to discourage outdated usage, and can be ignored.

In [ ]:
# from tensorflow.keras import Model

# MyGAN = Model(generator, discriminator)

# MyGAN.load_weights("/content/drive/MyDrive/data/volumetries/training_checkpoints/ckpt-7.index")


In [ ]:
# # #get the latest checkpoint file
# # checkpoint_dir = os.path.dirname(checkpoint_path)
# # latest = tf.train.latest_checkpoint(checkpoint_dir)

# latest="/content/drive/MyDrive/data/volumetries/training_checkpoints/ckpt-7.index"

# model_latest_checkpoint = make_three_dimentional_generator()
# # Load the previously saved weights
# model_latest_checkpoint.load_weights(latest)